In [5]:
using Cropbox

WebIO._IJuliaInit()

In [6]:
using CSV
using DataFrames

In [7]:
#parameter
C_parameters = @system Cultivar_parameters begin
   T_min ~ preserve(parameter)
   HI ~ preserve(parameter)
   I_50A ~ preserve(parameter)
   I_50B ~ preserve(parameter)
end

Cultivar_parameters

In [13]:
S_parameters = @system Species_parameters begin
    T_base ~ preserve(parameter)
    T_opt ~ preserve(parameter)
    RUE ~ preserve(parameter)
    I_50maxH ~ preserve(parameter)
    I_50maxW ~ preserve(parameter)
    T_max ~ preserve(parameter)
    T_ext ~ preserve(parameter)
    S_CO2 ~ preserve(parameter)
    S_water ~ preserve(parameter)
end

Species_parameters

In [15]:
#config
rice = @config(
    :Cultivar_parameters => (
        T_min = 2300,
        HI = 0.47,
        I_50A = 850,
        I_50B = 20
    ),
    :Species_parameters => (
        T_base = 9,
        T_opt = 26,
        RUE = 1.24,
        I_50maxH = 100,
        I_50maxW = 10,
        T_max = 34,
        T_ext = 50,
        S_CO2 = 0.08,
        S_water = 1.0
    )
)

T_min,=,2300
HI,=,0.47
I_50A,=,850
I_50B,=,20
T_base,=,9
T_opt,=,26
RUE,=,1.24
I_50maxH,=,100
I_50maxW,=,10
T_max,=,34
T_ext,=,50


In [17]:
#whether 불러오기
file_path = "weather.csv"  # 업로드된 파일 경로

whether_data = CSV.read(file_path, DataFrame)

@system Whether begin
end

Whether

In [21]:
#Phenology 구현
@system Phenology begin
    TT_Δ(T, T_base): added_temperature => begin
        T > T_base ? (T - T_base) : 0
        end ~ track(u"°C")

    TT(TT, TT_Δ): mean_temperature => (TT + TT_Δ) ~ accumulate
end

Phenology

In [84]:
@system Temperature_heat_drought_CO2 begin
    f_temp(T, T_base, T_opt): impact_temperature => begin
        if T < T_base
            0
        elseif T_base <= T && T < T_opt
            (T - T_base) /  (T_opt - T_base)
        else
            1
        end
    end ~ track

    f_heat(T_max, T_heat, T_extreme): impact_heat_stress => begin
        if T_max <= T_heat
            1
        elseif T_heat < T_max && T_max <= T_extreme
            1 - ((T_max - T_heat) / (T_extreme - T_heat))
        else
            0
        end
    end ~ track

    f_CO2(S_CO2, CO2): impact_CO2 => begin
        CO2 > 700 ? (1 + (S_CO2 * 350)) : (1 + S_CO2 * (CO2 - 350))
    end
    
end

LoadError: TypeError: in ##_Temperature#393, in type definition, expected Type, got a value of type Nothing